In [1]:
from tqdm import tqdm

import nltk
from nltk.corpus import gutenberg
from nltk import word_tokenize
from nltk import trigrams, ngrams

import string
myPunctation = string.punctuation +'“'+'”'+'-'+'’'+'‘'+'—'
myPunctation = myPunctation.replace('.', '')
myPunctation

from collections import defaultdict
import random

sentence_list = []

for Opera in tqdm(nltk.corpus.gutenberg.fileids()):
    #print(Opera)
    sentence_opera = gutenberg.sents(Opera)
    for sentence in sentence_opera:
        filteredSentence = [word for word in sentence if word not in myPunctation]
        sentence_list.append(" ".join(filteredSentence))

100%|██████████| 18/18 [00:28<00:00,  1.56s/it]


In [2]:
corpusText = " ".join(sentence_list)[:100]
corpusText

'Emma by Jane Austen 1816 VOLUME I CHAPTER I Emma Woodhouse handsome clever and rich with a comfortab'

In [10]:
#   wordTokens = word_tokenize(" ".join(sentence_list))
#   print("number of tokens: ", len(wordTokens))
#   print("number of unique tokens: ", len(set(wordTokens)))

number of tokens:  2644603
number of unique tokens:  50947


In [3]:
# Create a placeholder for model
trigram_model = defaultdict(lambda: defaultdict(lambda: 0))

# Count frequency of co-occurance
for sentence in sentence_list:
    sentence_words = word_tokenize(sentence)
    for w1, w2, w3 in trigrams(sentence_words, pad_right=True, pad_left=True):
        trigram_model[(w1, w2)][w3] += 1

In [4]:
list(trigram_model.keys())[:10]

[(None, None),
 (None, 'Emma'),
 ('Emma', 'by'),
 ('by', 'Jane'),
 ('Jane', 'Austen'),
 ('Austen', '1816'),
 ('1816', None),
 (None, 'VOLUME'),
 ('VOLUME', 'I'),
 ('I', None)]

In [5]:
trigram_model[("I", "think")]

defaultdict(<function __main__.<lambda>.<locals>.<lambda>()>,
            {'it': 58,
             'there': 12,
             'you': 46,
             'very': 4,
             'Mr': 3,
             'Harriet': 2,
             'a': 7,
             'they': 13,
             'her': 4,
             'one': 2,
             'Mrs': 1,
             'I': 57,
             'the': 17,
             'as': 2,
             'this': 3,
             'my': 3,
             '!': 1,
             'Emma': 1,
             'their': 1,
             'indeed': 1,
             'your': 1,
             'we': 22,
             'quite': 1,
             'she': 10,
             'will': 1,
             'who': 2,
             'of': 20,
             'he': 17,
             'Miss': 3,
             '.': 17,
             'at': 4,
             'on': 2,
             '?': 6,
             'than': 1,
             'must': 1,
             'do': 1,
             'would': 1,
             'but': 2,
             'before': 1,
             'him': 5,


In [6]:
# Let's transform the counts to probabilities
for w1_w2 in trigram_model:
    total_count = float(sum(trigram_model[w1_w2].values()))
    for w3 in trigram_model[w1_w2]:
        trigram_model[w1_w2][w3] /= total_count

In [7]:
float(sum(trigram_model[("I", "think")].values()))

0.999999999999998

In [8]:
# starting words
text = ["If", "I"]
sentence_finished = False
 
while not sentence_finished:
  # select a random probability threshold  
  r = random.random()
  accumulator = .0

  for word in trigram_model[tuple(text[-2:])].keys():
      accumulator += trigram_model[tuple(text[-2:])][word]
      # select words that are above the probability threshold
      if accumulator >= r:
          text.append(word)
          break

  #if text[-2:] == [None, None]:
  #    sentence_finished = True
  if text[-1] in [".", "!", "?"]:
      sentence_finished = True

print (' '.join([t for t in text if t]))

If I take it and peering in upon them .


### ngrams models

In [9]:
# Create a placeholder for model
fourgram_model = defaultdict(lambda: defaultdict(lambda: 0))

# Count frequency of co-occurance
for sentence in sentence_list:
    sentence_words = word_tokenize(sentence)
    for w1, w2, w3, w4 in ngrams(sentence_words, 4):
        fourgram_model[(w1, w2, w3)][w4] += 1

# Let's transform the counts to probabilities
for w1_w2_w3 in fourgram_model:
    total_count = float(sum(fourgram_model[w1_w2_w3].values()))
    for w4 in fourgram_model[w1_w2_w3]:
        fourgram_model[w1_w2_w3][w4] /= total_count

In [11]:
# starting words
text = ["If", "I",  "take"]
sentence_finished = False
 
while not sentence_finished:
  # select a random probability threshold  
  r = random.random()
  accumulator = .0

  for word in fourgram_model[tuple(text[-3:])].keys():
      accumulator += fourgram_model[tuple(text[-3:])][word]
      # select words that are above the probability threshold
      if accumulator >= r:
          text.append(word)
          break

  if text[-1] in [".", "!", "?"]:
      sentence_finished = True

print (' '.join([t for t in text if t]))

If I take the wings of a great deal too handsome in my opinion as to the Lord and to all of whom high or low Thy Selfe and Office deaftly show .
